In [221]:
# define a directory to save output files and data plots.
save_to_dir = '/Users/leilabelabassi/Documents/GitHub/MARACOOS/sos/'

In [62]:
from lxml.html import parse
from collections import OrderedDict

import requests
import json
import pandas as pd
import pyparsing as pp
import numpy as np

In [63]:
def list_catalog(html):
    
    doc = parse(html).getroot()
    list_st = []    
    for div in doc.cssselect('a'):
        list_i=div.text_content()
        list_st.append(list_i)
        
    return list_st

In [64]:
def parse_html(html):
    iname = []
    r = requests.get(html)
    word = pp.Word(pp.alphanums)

    s = r.text
    rule = pp.nestedExpr('<', '>')
    for match in rule.searchString(s):
        if len(match[0]) > 1:
            if match[0][1] == 'href=':
                NAME = match[0][2].split('catalog=')[-1]
                NAME = NAME.strip("'")
                
                if ".nc" in NAME:
                    iname.append(NAME)
                else:
                    if NAME.endswith('/thredds/catalog.html') == False | \
                       NAME.endswith('/thredds/serverInfo.html') == False | \
                       NAME.endswith('rucool.html') == False | \
                       NAME.endswith(".org") == False | \
                       NAME.endswith("index.html")== False | \
                       NAME.endswith("TDS.html")== False | \
                       NAME.startswith("http://www") == False:
                        iname.append(NAME)               
        

    return iname


In [146]:
def check_duplicates(liste):        # check for duplicates
    my_dict = {i:liste.count(i) for i in liste}
    double_entries = [i for i in my_dict if my_dict[i] > 1]
    if len(double_entries) > 0:
        print('duplicates exist:',double_entries)
    # remove duplicates
    liste = np.unique(liste)
    
    return liste
        

In [193]:
def check_nc(list_1, name):
    for iname in list_1:           
        num = 0
        if iname.startswith('/stable/dodsC/') | iname.startswith('/thredds/dodsC/'):
            num+=1
            anno = [str(len(list_1)) + ' type of data files exist with '+ str(num)+ ' as an OPENDAP type file']
            data_link = 'http://sos.maracoos.org' + iname

            df0 = pd.DataFrame({'annotation':[anno],
                            'data_link':[data_link[:-5]]},
                           index = [name])            
    return df0

In [67]:
global_list = parse_html('http://sos.maracoos.org/stable/agg_catalog'+'.html')#html_base[:-2]
global_list = check_duplicates(global_list)
global_list[:]

5 5
[]


array(['agg_catalogs/hrecos_agg_catalog.html',
       'agg_catalogs/mddnr_agg.html',
       'agg_catalogs/sldmb_agg_catalog.html',
       'agg_catalogs/surfrider_agg.html',
       'agg_catalogs/weatherflow_agg_catalog.html'], dtype='<U41')

In [218]:
#check what is inside the folders
g_name = global_list[3]
list_st = parse_html('http://sos.maracoos.org/stable/'+g_name)#html_base[:-13]global_list[0]
list_st = check_duplicates(list_st)
print(list_st)

['/thredds/catalog/surfrider_ag/catalog.html']


In [219]:
# seperate the list into categories
list_catalog = [hh for hh in list_st if hh.endswith('catalog.html') == True]
if len(list_catalog) >0:
    print('catalog', '\n',list_catalog)
list_dataset = [hh for hh in list_st if '?dataset=' in hh]
if len(list_dataset) >0:
    print('dataset', '\n',list_dataset)
list_nc = [hh for hh in list_st if '.nc' in hh]
if len(list_nc) >0:
    print('nc', '\n',list_nc)

catalog 
 ['/thredds/catalog/surfrider_ag/catalog.html']


In [217]:
# case of dataset
df= pd.DataFrame()
for name in list_st:

    html = 'http://sos.maracoos.org/stable/agg_catalogs/' + name
    list_i = parse_html(html)
    list_i = check_duplicates(list_i)
     
    # check for netcdf files
    list_nc = [hh for hh in list_i if '.nc' in hh]
    
    if len(list_nc)> 0:
        df0 = check_nc(list_nc, name)       
        df = df.append(df0)
        
filename = g_name.split('/')[-1].split('.html')[0]
df.to_csv(save_to_dir+filename+'.csv')

In [222]:
# case of catalog
g_name = global_list[3]
list_st = parse_html('http://sos.maracoos.org/stable/'+g_name)#html_base[:-13]global_list[0]
list_st = check_duplicates(list_st)

df = pd.DataFrame()
for name in list_st:

    html = 'http://sos.maracoos.org' + name
    list_i = parse_html(html)
    list_i = check_duplicates(list_i)
    
    
    # check for netcdf files
    list_nc = [hh for hh in list_i if '.nc' in hh]
    if len(list_nc)> 0:
        df0 = check_nc(list_nc, name)       
        df = df.append(df0)

    # check in folders for datasets
    list_html= [hh for hh in list_i if '.html' in hh]
    if len(list_html) > 0:
        for name_0 in list_html:

            html_0 = html[:-12] + name_0
            list_0 = parse_html(html_0)
            list_0 = check_duplicates(list_0)

            for name_1 in list_0:

                html_1 = html_0[:-12] + name_1
                list_1 = parse_html(html_1)
                list_1 = check_duplicates(list_1)

                list_folder = [hh for hh in list_1 if hh.endswith('catalog.html') == True]
                if len(list_folder) > 1:
                    print('problem, need to check new folders', list_folder)

                list_nc = [hh for hh in list_1 if '.nc' in hh]

                if len(list_nc) != 0:     
                    df0 = check_nc(list_nc, name)
                    df = df.append(df0)

filename = g_name.split('/')[-1].split('.html')[0]
df.to_csv(save_to_dir+filename+'.csv')

In [ ]:
empty_folder = []
html_link = []
folder_link = []
nc_in_folder = []

In [203]:
list_clean = [hh for hh in list_i if '%' not in hh]
list_html = [hh for hh in list_clean if '.html' in hh]
if len(list_html) > 0:
    print('.html', '\n', list_html)
list_nc = [hh for hh in list_clean if hh.endswith('.nc') == True]
if len(list_nc) > 0:
    print('.nc', '\n',list_nc)
list_ncml = [hh for hh in list_clean if hh.endswith('.ncml') == True]
if len(list_ncml) > 0:
    print('.ncml', '\n',list_ncml)

.html 
 ['/thredds/dodsC/sldmb/sldmb_99998_agg.ncml.html']
.ncml 
 ['/thredds/sos/sldmb/sldmb_99998_agg.ncml']


In [194]:
# check for netcdf files
list_nc = [hh for hh in list_i if '.nc' in hh]
df= pd.DataFrame()
if len(list_nc)> 0:
    df0 = check_nc(list_nc, name)       
    df = df.append(df0)

In [195]:
df

,annotation,data_link
sldmb_agg_catalog.html?dataset=sldmb_99998_agg,[2 type of data files exist with 1 as an OPENDAP type file],http://sos.maracoos.org/thredds/dodsC/sldmb/sldmb_99998_agg.ncml


In [169]:
for name in list_st:

    html = 'http://sos.maracoos.org/stable/agg_catalogs/' + name
    list_i = parse_html(html)
    list_i = check_duplicates(list_i)
    print(list_i)


['/thredds/dodsC/sldmb/sldmb_11090860_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_11090860_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_11090860_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_11090860_agg']
['/thredds/dodsC/sldmb/sldmb_11593180_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_11593180_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_11593180_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_11593180_agg']
['/thredds/dodsC/sldmb/sldmb_11596180_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_11596180_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_11596180_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_11596180_agg']
['/thredds/dodsC/sldmb/sldmb_11690950_agg.ncml.html'
 '/thred

['/thredds/dodsC/sldmb/sldmb_38645_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38645_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38645_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38645_agg']
['/thredds/dodsC/sldmb/sldmb_38646_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38646_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38646_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38646_agg']
['/thredds/dodsC/sldmb/sldmb_38649_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38649_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38649_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38649_agg']
['/thredds/dodsC/sldmb/sldmb_38650_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38650_agg.ncml'
 'ht

['/thredds/dodsC/sldmb/sldmb_38707_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38707_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38707_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38707_agg']
['/thredds/dodsC/sldmb/sldmb_38710_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38710_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38710_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38710_agg']
['/thredds/dodsC/sldmb/sldmb_38715_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38715_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38715_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38715_agg']
['/thredds/dodsC/sldmb/sldmb_38718_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38718_agg.ncml'
 'ht

['/thredds/dodsC/sldmb/sldmb_38782_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38782_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38782_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38782_agg']
['/thredds/dodsC/sldmb/sldmb_38784_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38784_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38784_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38784_agg']
['/thredds/dodsC/sldmb/sldmb_38786_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38786_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38786_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38786_agg']
['/thredds/dodsC/sldmb/sldmb_38794_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38794_agg.ncml'
 'ht

['/thredds/dodsC/sldmb/sldmb_38846_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38846_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38846_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38846_agg']
['/thredds/dodsC/sldmb/sldmb_38849_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38849_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38849_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38849_agg']
['/thredds/dodsC/sldmb/sldmb_38857_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38857_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38857_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38857_agg']
['/thredds/dodsC/sldmb/sldmb_38868_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38868_agg.ncml'
 'ht

['/thredds/dodsC/sldmb/sldmb_38930_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38930_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38930_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38930_agg']
['/thredds/dodsC/sldmb/sldmb_38935_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38935_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38935_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38935_agg']
['/thredds/dodsC/sldmb/sldmb_38936_agg.ncml.html'
 '/thredds/sos/sldmb/sldmb_38936_agg.ncml'
 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fagg_catalogs%2Fsldmb_agg_catalog.html&amp;dataset=sldmb_38936_agg'
 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.xml&amp;dataset=sldmb_38936_agg']


KeyboardInterrupt: 

In [ ]:
name

In [87]:
list_html= [hh for hh in list_i if '.html' in hh]
print(list_html)
list_nc = [hh for hh in list_i if '.nc' in hh]
print(list_nc)

['Aquarium_East_-_Bottom/catalog.html', 'Aquarium_East_-_Surface/catalog.html', 'Aquarium_West/catalog.html', 'Arundel_on_the_Bay/catalog.html', 'Bishopville_Prong/catalog.html', 'Budds_Landing/catalog.html', 'Camp_Tockwogh/catalog.html', 'Dares_Beach/catalog.html', 'Gooses_-_Bottom/catalog.html', 'Gooses_-_Surface/catalog.html', 'Greys_Creek/catalog.html', 'Harris_Creek_Downstream/catalog.html', 'Harris_Creek_Profiler/catalog.html', 'Harris_Creek_Upstream/catalog.html', 'Havre_de_Grace/catalog.html', 'Iron_Pot_Landing/catalog.html', 'Jug_Bay/catalog.html', 'Little_Monie_Creek/catalog.html', 'Masonville_Cove_Pier/catalog.html', 'Mataponi/catalog.html', 'Newport_Creek/catalog.html', 'Otter_Point_Creek/catalog.html', 'Pleasure_Island/catalog.html', 'Public_Landing/catalog.html', 'Riverside/catalog.html', 'Sandy_Pt._-_South_Beach/catalog.html', 'St_Georges_Creek/catalog.html', 'Susquahenna_Flats/catalog.html', 'Taylors_Island/catalog.html', 'Tilghman_Island/catalog.html', 'Tolchester/cata

In [143]:
html
html[:-12]

32


In [145]:
sos_df.to_csv(save_to_dir+'test.csv')

In [112]:
list_nc = [hh for hh in list_1 if '.nc' in hh]
print(list_nc)

['/stable/dodsC/mddnr_ag/Aquarium_East_-_Bottom/Aquarium_East_-_Bottom.ncml.html', '/stable/fileServer/mddnr_ag/Aquarium_East_-_Bottom/Aquarium_East_-_Bottom.ncml', '/stable/sos/mddnr_ag/Aquarium_East_-_Bottom/Aquarium_East_-_Bottom.ncml', 'http%3A%2F%2Fsos.maracoos.org%2Fstable%2Fcatalog%2Fmddnr_ag%2FAquarium_East_-_Bottom%2Fcatalog.html&amp;dataset=mddnr_ag%2FAquarium_East_-_Bottom%2FAquarium_East_-_Bottom.ncml', 'http://sos.maracoos.org/stable/catalog/mddnr_ag/Aquarium_East_-_Bottom/catalog.xml&amp;dataset=mddnr_ag/Aquarium_East_-_Bottom/Aquarium_East_-_Bottom.ncml']


In [121]:
df = pd.DataFrame()
sos_df = check_nc(list_nc, df)

In [122]:
sos_df.to_csv(save_to_dir+'test.csv')

In [156]:
global_list

array(['agg_catalogs/hrecos_agg_catalog.html',
       'agg_catalogs/mddnr_agg.html',
       'agg_catalogs/sldmb_agg_catalog.html',
       'agg_catalogs/surfrider_agg.html',
       'agg_catalogs/weatherflow_agg_catalog.html'], dtype='<U41')

In [159]:
# this routine works for hrecos&weatherflow

df = pd.DataFrame()

list_st = parse_html(html_base[:-13]+global_list[0])
list_st = check_duplicates(list_st)

for name in list_st:

    html = html_base + name

    list_i = parse_html(html)
    list_i = check_duplicates(list_i)

    if len(list_i) != 0:       
        list_nc = [hh for hh in list_i if '.nc' in hh]
        if len(list_nc) == 0:
            for iname in list_nc:           
                num = 0
                if iname.startswith('/stable/dodsC/'):
                    num+=1
                    anno = [str(len(list_nc)) + ' type of data files exist with '+ str(num)+ ' as an OPENDAP type file']
                    data_link = html_base[:-13] + iname

                    df0 = pd.DataFrame({'annotation':[anno],
                                    'data_link':[data_link[:-5]]},
                                   index = [name])
                    df = df.append(df0)
                else:
                    if iname.endswith('catalog.html'):
                        print('subfolder exist')

duplicates exist: ['hrecos_agg_catalog.html?dataset=stationHRTVBM-agg', 'hrecos_agg_catalog.html?dataset=stationHRWSTPTH-agg']


In [ ]:
pd.set_option('display.max_colwidth', None)                    
print(df['annotation'])

In [160]:
filename = global_list[0].split('/')[-1].split('.html')[0]
df.to_csv(save_to_dir+filename+'.csv')

In [ ]:
list_st = list_catalog(html_base)
print(list_st)
list_st = [x for x in list_st if 'Aggregated Data' in x]
list_st = [x.split(' Aggregated Data')[0].split('Station ')[-1] for x in list_st]

my_dict = {i:list_st.count(i) for i in list_st}
my_dict

In [ ]:
html_base = 'http://sos.maracoos.org/stable/agg_catalogs/sldmb_agg_catalog.html'

list_st = list_catalog(html_base)
    
list_st = [x for x in list_st if 'Aggregation' in x]
list_st = [x.split(' Aggregation')[0].split('SLDBM Station ')[-1] for x in list_st]
print(len(list_st))
my_dict = {i:list_st.count(i) for i in list_st}
my_dict

In [ ]:
html_base = 'http://sos.maracoos.org/stable/agg_catalogs/weatherflow_agg_catalog.html'
list_st = list_catalog(html_base)

list_st = [x for x in list_st if 'Aggregated' in x]
list_st = [x.split(' Aggregated')[0].split('Station-')[-1] for x in list_st]
print(len(list_st))
my_dict = {i:list_st.count(i) for i in list_st}
OrderedDict(sorted(my_dict.items(), key=lambda t: t[0]))

In [ ]:
#  'http://sos.maracoos.org/thredds/catalog/mddnr_ag/catalog.html'
html_base = 'http://sos.maracoos.org/thredds/catalog/mddnr_ag/'
html_valid = 'catalog.html'
html_whole = html_base + html_valid

In [ ]:
list_st = list_catalog(html_whole)

list_st = [x for x in list_st if '/' in x]
list_st

In [ ]:
#http://sos.maracoos.org/thredds/catalog/mddnr_ag/Williston_Lake_Pier/catalog.html

In [ ]:
total = 0
for name in list_st:
    html_file = html_base + name + html_valid
    doc = parse(html_file).getroot()
    list_nc = []    
    for div in doc.cssselect('a'):
        list_i=div.text_content()
        list_nc.append(list_i)
    
    list_nc = [x for x in list_nc if '.nc' in x]
    total = total+len(list_nc)
    print(name, len(list_nc), total)
print(total)